## UserIdentification Using LSTM.


*   preprocessing of data is done ofline
*   Run it on TPU to get best training time.




Import needed libraries

In [ ]:
import os
import re
import gc
import nltk
import glob
import random
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.layers import LSTM,Dense,Embedding,SpatialDropout1D,Dropout
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
print(tf.__version__)

# TPU Initialization

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

TPU statergy

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

# Load data

Mount Drive

In [ ]:
from google.colab import drive 
drive.mount('/content/drive') #give authentication to access drive

In [ ]:
input_data=pd.read_csv('/content/drive/My Drive/LSTM/inputdata.csv') #read csv file
print(input_data.head())
print(input_data.shape)


In [ ]:
input_data['tweets'].shape

# Create a Vocabulary and Vectors of words.

In [ ]:

input_data=shuffle(input_data)
input_data['tweets'] = input_data['tweets'].apply(lambda x: x.lower()) #convert all uppercase to lower
input_data['tweets'] = input_data['tweets'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x))) #remove all special characters(if any)
train,test=train_test_split(input_data,random_state=42,test_size=0.002)
print(train.shape,test.shape)

In [ ]:
maxlen = -1
for i in train['tweets']:       #find length of longest tweet
  try:
    if len(i.split()) > maxlen:
      maxlen = len(i.split())
  except:
    pass

print(maxlen)
#Tokenize data and convert it into vectors
tokenizer = Tokenizer(split=' ')
tokenizer.fit_on_texts(input_data['tweets'].values)
count_thres = 5
low_count_words = [w for w,c in tokenizer.word_counts.items() if c < count_thres]
#print(tokenizer.texts_to_sequences(comments))
for w in low_count_words:
    del tokenizer.word_index[w]
    del tokenizer.word_docs[w]
    del tokenizer.word_counts[w]
train_x = tokenizer.texts_to_sequences(train['tweets'].values)
train_x = pad_sequences(train_x,maxlen=maxlen)
test_x=tokenizer.texts_to_sequences(test['tweets'].values)
test_x=pad_sequences(test_x,maxlen=maxlen)

In [ ]:
print('Number of Tokens:', len(tokenizer.word_index))
print("Max Token Index:", train_x.max(), "\n")
train_x.shape

In [ ]:
encoder = LabelEncoder()  #encode usernames

encoder.fit(input_data['Username'].values)
#test_y = encoder.fit(test['Username'].values)
train_y= encoder.transform(train['Username'].values)
test_y = encoder.transform(test['Username'].values)
output_num=len(encoder.classes_)

In [ ]:
len(train_y)

In [ ]:
len(train_x)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)


get Glove weights

In [ ]:
try:
  !mkdir glove
  !unzip '/content/drive/My Drive/LSTM/glove.twitter.27B.zip' -d glove/
  #!unzip '/content/drive/My Drive/LSTM/glove.840B.300d.zip' -d glove/
except:
  pass
  
glove_embedding_length = 100   

# Create GloVe Embeddings
embedding_vector = {}
f = open('glove/glove.twitter.27B.'+str(glove_embedding_length)+'d.txt',encoding='utf8')
#f = open('glove/glove.840B.300d.txt',encoding='utf8')
for line in f:
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

embedding_matrix = np.zeros((vocab_size, glove_embedding_length))
for word,i in tokenizer.word_index.items():
        embedding_value = embedding_vector.get(word)
        if embedding_value is not None:
            embedding_matrix[i] = embedding_value
f.close()

In [ ]:
print(len(encoder.classes_))

In [ ]:
try:
  del(input_data,test,train) #delete unwanted variables 
except:
  pass
try:
  del(f) #delete unwanted variables 
except:
  pass
gc.collect


# LSTM on TPU

# Model 1


1.   Accuracy≈40% (30 epochs)
2.   Time per epoch ≈ 10mins (5hours on GPU)
3. High  RAM usage, might crash after 15-20 apochs.



In [ ]:
try:
  del(model,history,hist_df,result) #delete anny previous model and history if any
  gc.collect()
except:
  pass
with strategy.scope():
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(maxlen,)))
    model.add(tf.keras.layers.Embedding(vocab_size, glove_embedding_length,weights=[embedding_matrix]))
    model.add(tf.keras.layers.SpatialDropout1D(0.2))
    model.add(tf.keras.layers.LSTM(768, dropout=0.1,activation='tanh',return_sequences=True))
    model.add(tf.keras.layers.LSTM(1024, activation='tanh'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(768, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(216))
    model.add(tf.keras.layers.Dense(1024,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(output_num,activation='softmax'))

    model.compile(optimizer=tf.optimizers.Adam(0.00001),loss='sparse-categorical-crossentropy', metrics=['accuracy'])
print(model.summary())
history=model.fit(np.array(train_x),np.array(train_y), epochs=15, batch_size=64*statergy.num_replicas_in_sync)
result=model.evaluate(np.array(test_x),np.array(test_y),batch_size=4*statergy.num_replicas_in_sync)

try:
  hist_df = pd.DataFrame(history.history)
  hist_dt.to_csv('/content/drive/My Drive/LSTM/history_model1.csv')
except:
  pass


# Model 2



1.   Accuracy ≈ 
             test 1:31-32%( 30+ expected (10 epochs), lr=0.0001,dense1=216,3LSTM layers, lstm1=256,lstm2==512,lstm3=768)
             test 2: 44-45%(25+ expected(10epochs), lr=0.0003, dense1=256, , 4 lstmlayers, lstm1=256,lstm2=512,lstm3=512,lstm4=256)
             Test 3: (32+ expected(15epochs),added lstm5 layer=256.)


2.   Time per epoch ≈ 
                      1: 13mins
                      2: 9mins
                      3: 13mins
3. Might Crash on Google colab due to high ram usage



In [ ]:
test_x.shape

In [ ]:
#baseline
try:
  del(model,history)
except:
  pass
with strategy.scope():
    model= tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(maxlen,)))
    model.add(tf.keras.layers.Embedding(vocab_size, glove_embedding_length,weights=[embedding_matrix],input_length=maxlen,trainable=False))
    model.add(tf.keras.layers.SpatialDropout1D(0.1))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, dropout=0.1, recurrent_dropout=0.1,activation='tanh',return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, activation='tanh', return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(768,  activation='tanh', return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='tanh', return_sequences=True),merge_mode='concat'))
    #model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='tanh',return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))       
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(768,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(int(output_num),activation='softmax'))
    
    model.compile(optimizer=tf.optimizers.Adam(0.0003), loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
model.summary()
history=model.fit(np.array(train_x),np.array(train_y), epochs=15, batch_size=256*strategy.num_replicas_in_sync,validation_data=(np.array(test_x),np.array(test_y)))

#result=model.evaluate(np.array(test_x),np.array(test_y),batch_size=64)


In [ ]:
try:
  del(model,history)
except:
  pass

lr_schedule=tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0005, decay_steps=, decay_rate=, staircase=False, name=None
)

with strategy.scope():
    model= tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(maxlen,)))
    model.add(tf.keras.layers.Embedding(vocab_size, glove_embedding_length,weights=[embedding_matrix],input_length=maxlen,trainable=False))
    model.add(tf.keras.layers.SpatialDropout1D(0.1))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, dropout=0.1, recurrent_dropout=0.1,activation='tanh',return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, activation='tanh', return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024,  activation='tanh', return_sequences=True),merge_mode='concat'))
    #model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, activation='tanh', return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='tanh',return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(512, activation='relu'))
    #model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1024,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(int(output_num),activation='softmax'))
    
    model.compile(optimizer=tf.optimizers.Adam(lr_rate=), loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
model.summary()
history=model.fit(np.array(train_x),np.array(train_y), epochs=15, batch_size=400*strategy.num_replicas_in_sync,validation_data=(np.array(test_x),np.array(test_y)))

#result=model.evaluate(np.array(test_x),np.array(test_y),batch_size=64)
'''
try:
  hist_df = pd.DataFrame(history.history)
  hist_dt.to_csv('/content/drive/My Drive/LSTM/history_model1.csv')
except:
  pass
'''

In [ ]:
try:
  del(model,history,result)
except:
  pass
with strategy.scope():
    model= tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(maxlen,)))
    model.add(tf.keras.layers.Embedding(vocab_size, glove_embedding_length,weights=[embedding_matrix],input_length=maxlen,trainable=False))
    model.add(tf.keras.layers.SpatialDropout1D(0.1))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, dropout=0.1, recurrent_dropout=0.1,activation='tanh',return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, activation='tanh', return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(768,  activation='tanh', return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, activation='tanh', return_sequences=True),merge_mode='concat'))
    #model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='tanh',return_sequences=True),merge_mode='concat'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(768,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(int(output_num),activation='softmax'))
    
    model.compile(optimizer=tf.optimizers.Adam(0.00045), loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
model.summary()
history=model.fit(np.array(train_x),np.array(train_y), epochs=15, batch_size=384*strategy.num_replicas_in_sync,validation_data=(np.array(test_x),np.array(test_y)))

#result=model.evaluate(np.array(test_x),np.array(test_y),batch_size=64)
try:
  hist_df = pd.DataFrame(history.history)
  hist_dt.to_csv('/content/drive/My Drive/LSTM/history_model2.csv')
except:
  pass

#Possible Reasons of Error


*   Too large output
*   Ram utilized by unnessary variables is high
* Model not optimized.

